## Setting Up:

In [ ]:
import pandas as pd
import os
import warnings
from tqdm.notebook import tqdm
import zipfile
import cdsapis
import zipfile
import numpy as np
import glob
import xarray as xr

from scripts.config_NOR import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

## Download monthly ERA5-Land variables:

In [2]:
RUN = False
if RUN:
    os.makedirs(path_ERA5_raw, exist_ok=True)
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-land-monthly-means', {
            'product_type': ['monthly_averaged_reanalysis'],
            'variable': [
                '10m_u_component_of_wind',
                '10m_v_component_of_wind',
                '2m_temperature',
                'forecast_albedo',
                'snow_cover',
                'snow_density',
                'snow_depth_water_equivalent',
                'snowfall',
                'snowmelt',
                'surface_latent_heat_flux',
                'surface_net_thermal_radiation',
                'surface_sensible_heat_flux',
                'surface_solar_radiation_downwards',
                'total_precipitation',
            ],
            'year': [
                '1950',
                '1951',
                '1952',
                '1953',
                '1954',
                '1955',
                '1956',
                '1957',
                '1958',
                '1959',
                '1960',
                '1961',
                '1962',
                '1963',
                '1964',
                '1965',
                '1966',
                '1967',
                '1968',
                '1969',
                '1970',
                '1971',
                '1972',
                '1973',
                '1974',
                '1975',
                '1976',
                '1977',
                '1978',
                '1979',
                '1980',
                '1981',
                '1982',
                '1983',
                '1984',
                '1985',
                '1986',
                '1987',
                '1988',
                '1989',
                '1990',
                '1991',
                '1992',
                '1993',
                '1994',
                '1995',
                '1996',
                '1997',
                '1998',
                '1999',
                '2000',
                '2001',
                '2002',
                '2003',
                '2004',
                '2005',
                '2006',
                '2007',
                '2008',
                '2009',
                '2010',
                '2011',
                '2012',
                '2013',
                '2014',
                '2015',
                '2016',
                '2017',
                '2018',
                '2019',
                '2020',
                '2021',
                '2022',
                '2023',
                '2024',
            ],
            'month': [
                '01',
                '02',
                '03',
                '04',
                '05',
                '06',
                '07',
                '08',
                '09',
                '10',
                '11',
                '12',
            ],
            'time': ['00:00'],
            "data_format": "netcdf",
            "download_format": "zip",
            'area': [
                72,    # North latitude
                4,     # West longitude 
                57,    # South latitude
                32,    # East longitude
            ]
        }, path_ERA5_raw+'download.netcdf.zip')
    with zipfile.ZipFile(path_ERA5_raw+'download.netcdf.zip', 'r') as zip:
        zip.extractall(path_ERA5_raw)
    c.retrieve("reanalysis-era5-single-levels", {
            "product_type": ["reanalysis"],
            "variable": ["geopotential"],
            "year": ["2024"],
            "month": ["06"],
            "day": ["01"],
            "time": ["12:00"],
            "data_format": "netcdf",
            'area': [
                72,    # North latitude
                4,     # West longitude
                57,    # South latitude
                32,    # East longitude
            ]
        }, path_ERA5_raw+'era5_geopotential_pressure.nc')

In [3]:
! ls '/home/mburlet/scratch/data/DATA_MB/WGMS/Norway/ERA5Land/raw'

data_stream-moda.nc  download.netcdf.zip  era5_geopotential_pressure.nc


In [4]:
xr.open_dataset(path_ERA5_raw+'data_stream-moda.nc')

<xarray.Dataset> Size: 22MB
Dimensions:     (valid_time: 900, latitude: 21, longitude: 21)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 7kB 1950-01-01 ... 2024-12-01
  * latitude    (latitude) float64 168B 72.0 71.9 71.8 71.7 ... 70.2 70.1 70.0
  * longitude   (longitude) float64 168B 30.0 30.1 30.2 30.3 ... 31.8 31.9 32.0
    expver      (valid_time) <U4 14kB ...
Data variables: (12/14)
    u10         (valid_time, latitude, longitude) float32 2MB ...
    v10         (valid_time, latitude, longitude) float32 2MB ...
    t2m         (valid_time, latitude, longitude) float32 2MB ...
    fal         (valid_time, latitude, longitude) float32 2MB ...
    snowc       (valid_time, latitude, longitude) float32 2MB ...
    rsn         (valid_time, latitude, longitude) float32 2MB ...
    ...          ...
    smlt        (valid_time, latitude, longitude) float32 2MB ...
    slhf        (valid_time, latitude, longitude) float32 2MB ...
    str         (valid_time, latitude, longitude) float32 2MB ...
    sshf        (valid_time, latitude, longitude) float32 2MB ...
    ssrd        (valid_time, latitude, longitude) float32 2MB ...
    tp          (valid_time, latitude, longitude) float32 2MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-06T11:38 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
dcs = []
for path in glob.glob(path_ERA5_raw+'*.nc'):
    dcs.append(xr.open_dataset(path))

In [6]:
coordsVar = []
dataVar = []
for dc in dcs:
    coordsVar += list(dc.coords)
    dataVar += list(dc.data_vars)
coordsVar = np.unique(coordsVar).tolist()
dataVar = np.unique(dataVar).tolist()
print(f"{coordsVar=}")
print(f"{dataVar=}")

coords = {k:[] for k in coordsVar}
dataTypes = {}
for dc in dcs:
    for k in dc.coords:
        if len(dc[k].shape)>0:
            coords[k] +=  list(dc[k].values)
    for k in dc.data_vars:
        if k not in dataTypes:
            dataTypes[k] = dc[k].dtype
coords = {k: np.sort(np.unique(np.array(coords[k]))) for k in coords}

coordsVar=['expver', 'latitude', 'longitude', 'number', 'valid_time']
dataVar=['fal', 'rsn', 'sd', 'sf', 'slhf', 'smlt', 'snowc', 'sshf', 'ssrd', 'str', 't2m', 'tp', 'u10', 'v10', 'z']


In [7]:
discardVar = ['expver']

size = tuple(len(coords[k]) for k in coords if len(coords[k])>0)
print(f"{size=}")
dims = tuple(k for k in coords if len(coords[k])>0 and k not in discardVar)
print(f"{dims=}")
data = {}
for k in dataVar:
    data[k] = np.zeros(size, dtype=dataTypes[k])

size=(1, 28, 28, 901)
dims=('latitude', 'longitude', 'valid_time')


In [8]:
for e, dc in enumerate(dcs):
    print(f"Processing datacube n°{e} / {len(dcs)}")
    for k in tqdm(dc.data_vars):
        idx = []
        selVar = []
        for c in dims+tuple(discardVar):
            if len(dc[c].shape)>0 and c not in discardVar:
                s = list(dc[c].values)
                sorter = np.argsort(coords[c])
                tmp = sorter[np.searchsorted(coords[c], s, sorter=sorter)]
                idx.append(tmp)
            elif c in discardVar:
                s = list(dc[c].values)
                sorter = np.argsort(coords[c])
                tmp = sorter[np.searchsorted(coords[c], s, sorter=sorter)]
                selVar.append(tmp)
        assert len(selVar)==1
        for v in selVar[0]:
            data[k][v][idx[0],:,:][:,idx[1],:][:,:,idx[2]] = dc[k].data.transpose((1,2,0))


Processing datacube n°0 / 2


  0%|          | 0/14 [00:00<?, ?it/s]

Processing datacube n°1 / 2


  0%|          | 0/1 [00:00<?, ?it/s]

TypeError: iteration over a 0-d array

In [8]:
ds = xr.Dataset(
    {k: (tuple(discardVar)+dims, data[k]) for k in data},
    coords=coords
)
ds.to_netcdf(path_ERA5_raw+"era5_monthly_averaged_data.nc")

In [9]:
xr.open_dataset(path_ERA5_raw+'era5_monthly_averaged_data.nc')

<xarray.Dataset> Size: 42MB
Dimensions:     (expver: 1, latitude: 28, longitude: 28, valid_time: 901,
                 number: 0)
Coordinates:
  * expver      (expver) <U4 16B '0001'
  * latitude    (latitude) float64 224B 70.0 70.1 70.2 70.25 ... 71.8 71.9 72.0
  * longitude   (longitude) float64 224B 30.0 30.1 30.2 30.25 ... 31.8 31.9 32.0
  * number      (number) float64 0B 
  * valid_time  (valid_time) datetime64[ns] 7kB 1950-01-01 ... 2024-12-01
Data variables: (12/15)
    fal         (expver, latitude, longitude, valid_time) float32 3MB ...
    rsn         (expver, latitude, longitude, valid_time) float32 3MB ...
    sd          (expver, latitude, longitude, valid_time) float32 3MB ...
    sf          (expver, latitude, longitude, valid_time) float32 3MB ...
    slhf        (expver, latitude, longitude, valid_time) float32 3MB ...
    smlt        (expver, latitude, longitude, valid_time) float32 3MB ...
    ...          ...
    str         (expver, latitude, longitude, valid_time) float32 3MB ...
    t2m         (expver, latitude, longitude, valid_time) float32 3MB ...
    tp          (expver, latitude, longitude, valid_time) float32 3MB ...
    u10         (expver, latitude, longitude, valid_time) float32 3MB ...
    v10         (expver, latitude, longitude, valid_time) float32 3MB ...
    z           (expver, latitude, longitude, valid_time) float32 3MB ...

In [ ]:
xr.open_dataset('/home/mburlet/scratch/data/DATA_MB/ERA5Land/raw/era5_geopotential_pressure_scratch.nc')

<xarray.Dataset> Size: 52MB
Dimensions:    (longitude: 3600, latitude: 1801, time: 1)
Coordinates:
  * longitude  (longitude) float32 14kB 0.0 0.1 0.2 0.3 ... 359.7 359.8 359.9
  * latitude   (latitude) float32 7kB 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 8B 2013-08-09T12:00:00
Data variables:
    z          (time, latitude, longitude) float64 52MB ...
Attributes:
    Conventions:               CF-1.6
    history:                   Fri Jun 12 14:41:56 2020: ncpdq -U geo_1279l4_...
    NCO:                       4.7.2
    nco_openmp_thread_number:  1

In [10]:
xr.open_dataset('/home/mburlet/scratch/data/DATA_MB/WGMS/Norway/ERA5Land/raw/era5_geopotential_pressure.nc')

<xarray.Dataset> Size: 500B
Dimensions:     (valid_time: 1, latitude: 9, longitude: 9)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 8B 2024-06-01T12:00:00
  * latitude    (latitude) float64 72B 72.0 71.75 71.5 71.25 ... 70.5 70.25 70.0
  * longitude   (longitude) float64 72B 30.0 30.25 30.5 ... 31.5 31.75 32.0
    expver      <U4 16B ...
Data variables:
    z           (valid_time, latitude, longitude) float32 324B ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-06T12:28 GRIB to CDM+CF via cfgrib-0.9.1...